In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("london.csv")

In [6]:
# check null values
df.isna().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

In [12]:
# check shape
df.shape

(17414, 10)

In [13]:
# check types
df.dtypes

timestamp        object
cnt               int64
t1              float64
t2              float64
hum             float64
wind_speed      float64
weather_code    float64
is_holiday      float64
is_weekend      float64
season          float64
dtype: object

In [35]:
# convert date types
df.timestamp = pd.to_datetime(df.timestamp)
df.is_holiday = df.is_holiday.astype('int')
df.is_weekend = df.is_weekend.astype('int')
df.season = df.season.astype('int')
df.weather_code = df.weather_code.astype('int')